In [5]:
from util import *
from seq2seq import *
import tltk, jiwer

# additional dict data

In [4]:

df = pd.read_csv('data/g2p_dict.csv')
df.g.to_csv('data/dic_x.txt', index=False, header=None)
df.rtgs.to_csv('data/dic_y.txt', index=False, header=None)

# load data

In [6]:
# load data
train_x = open('data/train_x.txt').read().splitlines()
train_y = open('data/train_y.txt').read().splitlines()
test_x = open('data/test_x.txt').read().splitlines()
test_y = open('data/test_y.txt').read().splitlines()

dic_x = open('data/dic_x.txt').read().splitlines()
dic_y = open('data/dic_y.txt').read().splitlines()

In [7]:
# Instantiation
data = Data()
data.set_train(train_x, train_y)
data.set_test(test_x, test_y)

data.set_train(dic_x, dic_y)

In [8]:
# build model
seq2seq1 = Seq2Seq(data, attention=False)
seq2seq1.build(show_summary=False, opt='rmsprop', loss='categorical_crossentropy')

seq2seq2 = Seq2Seq(data, attention=True)
seq2seq2.build(show_summary=False, opt='rmsprop', loss='categorical_crossentropy')

In [10]:
# train
seq2seq1.train(batch_size=100, epoch=10, dev_split=0)
seq2seq2.train(batch_size=100, epoch=10, dev_split=0)

Epoch 1/10
301/301 [==============================] - 224s 745ms/step - loss: 0.2143 - accuracy: 0.5346
Epoch 2/10
301/301 [==============================] - 355s 1s/step - loss: 0.1057 - accuracy: 0.7623
Epoch 3/10
301/301 [==============================] - 377s 1s/step - loss: 0.0682 - accuracy: 0.8453
Epoch 4/10
235/301 [======================>.......] - ETA: 1:19 - loss: 0.0513 - accuracy: 0.8837

In [26]:
pred1 = seq2seq1.predict(join_by='')
pred2 = seq2seq2.predict(join_by='')
df = pd.DataFrame({'thai':test_x,'pred1':pred1,'pred2':pred2, 'answer':test_y})
pred_tltk = [tltk.nlp.th2roman(x).replace('<s/>','').replace(' ','') for x in list(df.answer)]
df['tltk'] = pred_tltk
df.to_csv('pred_withdic.csv', index=False)

100%|██████████| 810/810 [03:51<00:00,  3.51it/s]


In [29]:
def evaluate(df):
    cols = df.columns
    pred_list = list(df.pred1)
    gold_list = list(df.answer)
    tempdf = pd.DataFrame(index=['WER','CER macro', 'CER micro'])

    for col in cols:
        if col in ['answer', 'thai']:
            continue
        pred_list = list(df[col])
        WER = jiwer.wer(pred_list, gold_list)
        CERmacro = np.mean([jiwer.cer(p,g) for p,g in zip(pred_list, gold_list)])
        CERmicro = sum([jiwer.cer(p,g)*len(g) for p,g in zip(pred_list, gold_list)])/sum([len(g) for g in gold_list])
        tempdf[col] = [WER, CERmacro, CERmicro]
    return tempdf

evaluate(df)

,pred1,pred2,tltk
WER,0.259259,0.156790,0.101235
CER macro,0.075115,0.038668,0.013554
CER micro,0.075955,0.038693,0.013101


In [18]:
len(pred_tltk)

810